In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from __future__ import print_function, division
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
import pandas as pd
import seaborn.apionly as sns
import matplotlib.ticker as ticker
import math
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
import random as rand
# from sys import maxint
import scipy.stats as ss
import itertools
from itertools import islice, takewhile
import operator
from scipy.stats import t, cauchy, norm
from functools import reduce
import scipy
import pandas as pd

/home/mkhoshle/miniconda3/envs/research/lib/python3.5/site-packages/seaborn/apionly.py:6: UserWarning: As seaborn no longer sets a default style on import, the seaborn.apionly module is deprecated. It will be removed in a future version.
  warnings.warn(msg, UserWarning)


## States: 0:Inactive, 1:Active, 2:Dark, 3:Photo-Bleached

In [2]:
def transition_matrix(p):
    Q = np.array([[-0.5,0.5,0,0],[0,-11.0,10.0,1.0],[0,0.1,-0.1,0],[0,0,0,0]])     # Rate Matrix (Hz)
    pi = scipy.linalg.expm(Q*np.float64(p))     
    # Transition Matrix exp(Qdt)        
    return pi # Unitless

In [3]:
def get_signal(pi,partitions):
    states = []
    state = 0 
    states.append(state)
    for i in range(1,len(partitions)-1):
        state = np.random.choice(4,p=pi[state])
#       state = 0 if i==0 else 3 if i==len(partitions)-1 else np.random.choice(4,p=pi[state_old])   
        states.append(state)   
    
    return states

In [4]:
def Generate_syntethic_data(step,pi,partitions,B):                                               
    n_microstates = 4                                     # Total number of microstates (I,A,D,B)
    S = []                                                # State trajectory
    for i in range(B):
        print(i)
        S.append(get_signal(pi,partitions))
        
    S = pd.DataFrame(S)  
    Cs = np.where(S==1, 1,0)
    Cs = np.sum(Cs, axis=0)                               # Number of active particles vs time    
    N = np.shape(S)[1]                                    # Number of time levels
    # Calculate Observations (Noise)
    Miu_back = 2e+4                                       # Background photons emission rate (photon/s)
    Miu = pd.DataFrame(index=range(B),columns=range(N))   # Emission rate for each particle at time level n (photon/s)
    miu_bright = 1e+5                                     # Emission rate at the Bright state (photon/s)
    miu_dark = 0.0                                        # Emission rate at the Dark state (photon/s)
    matches = S[(S==1)]
    # Get emission rate for each particle in time
    Miu = np.where(matches==1,miu_bright,miu_dark)        # Change in emission rate for each particle over time 
    tn = 0.95*step                                        # Total exposure time (s)
    Wn = pd.Series(ss.poisson.rvs(mu=(tn*(Miu_back+Miu.sum(axis=0)))))
    return S, Cs, Wn, Miu

In [5]:
# Our time step is uniform: 
B = 1      # Number of fluorphored particles
step = 0.01 # [S]
partitions = [i*step for i in range(round(200./step))]
# partitions = np.random.uniform(0,0.001,400000)

In [6]:
pi = transition_matrix(step)

In [7]:
pi

array([[9.95012479e-01, 4.72285706e-03, 2.40596046e-04, 2.40677016e-05],
       [0.00000000e+00, 8.95880600e-01, 9.46496180e-02, 9.46978182e-03],
       [0.00000000e+00, 9.46496180e-04, 9.99048684e-01, 4.82001787e-06],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [ ]:
S, Cs, Wn, Miu = Generate_syntethic_data(step,pi,partitions,B)

In [ ]:
S.to_csv('trajectory.csv')

In [ ]:
t = partitions

In [ ]:
Wnm = pd.DataFrame([ss.poisson.rvs(mu=(tn*(Miu[m,:]))) for m in range(B)])

In [ ]:
fig = plt.figure(figsize=(10,16)) 
plt.style.use('ggplot')
sns.set_style('ticks')
plt.rcParams['font.size'] = 12

n = 0
for m in range(B):
    ax = fig.add_subplot(5,3,n+1)
    plt.scatter(t[:-1],Wnm.iloc[m,:]) 
#         if n % 3 == 0:
#             ax.set_ylabel('State')
#         if n>11:
#             ax.set_xlabel('time (s)')
    ax.set_title(r'$particle-ID: {}$'.format(str(m)))
    ax.legend().set_visible(False)
    sns.despine(offset=12,ax=ax,trim=True)
    n = n+1

# plt.subplots_adjust(top=0.92,bottom=0.08,left=0.1,right=0.95,wspace=0.6,hspace=0.6)
plt.tight_layout()
plt.show() 

In [ ]:
def plot_state(df):
    fig = plt.figure(figsize=(10,16)) 
    plt.style.use('ggplot')
    sns.set_style('ticks')
    plt.rcParams['font.size'] = 12

    n = 0
    for i in range(len(df)):
        ax = fig.add_subplot(5,3,n+1)
        plt.step(x=t[:-1],y=S.iloc[i,:])
        if n % 3 == 0:
            ax.set_ylabel('State')
        if n>11:
            ax.set_xlabel('time (s)')
        ax.set_title(r'$particle-ID: {}$'.format(str(i)))
        ax.legend().set_visible(False)
        sns.despine(offset=12,ax=ax,trim=True)
        n = n+1

    # plt.subplots_adjust(top=0.92,bottom=0.08,left=0.1,right=0.95,wspace=0.6,hspace=0.6)
    plt.tight_layout()
    plt.show() 
    return None

In [ ]:
np.shape(S.iloc[0,:])

In [ ]:
plt.step(x=t[:12000],y=S.iloc[0,:12000])

In [ ]:
plot_state(S)

In [ ]:
def get_plot(df,xlabel,ylabel):
    fig = plt.figure(figsize=(4,4)) 
    plt.style.use('ggplot')
    sns.set_style('ticks')
    plt.rcParams['font.size'] = 12
    plt.plot(t[:-1],df)
    plt.ylabel(ylabel)
    plt.xlabel(xlabel)
    plt.tight_layout()
    sns.despine(offset=10)
    return None

In [ ]:
get_plot(Cs,'time (S)','$N^{Active}_{Particles}$')

In [ ]:
get_plot(Wn,'time (s)','$N^{Emitted}_{Photons}$')

In [ ]:
len(t[:-1])

In [ ]:
out = pd.concat([pd.Series(t[:-1]),Wn],axis=1)

In [ ]:
out.to_csv('observation.csv')

In [ ]:
mu = [tn[n]*(Miu_back+Miu.sum(axis=0)[n]) for n in range(N)]

In [ ]:
x = plt.hist(Wn,bins=100)
# Y = plt.hist(mu, bins=100)
plt.ylim([0,20])